In [1]:
# TODO: use manual HTML features
import numpy as np
import pandas as pd
from utils import run_ML
from sklearn.metrics import f1_score
from itertools import groupby
import pickle
from pathlib import Path
from sklearn.model_selection import StratifiedKFold
import os

/home/vanhoan310/miniconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import codecs
import gensim
import urllib
import logging
from pathlib import Path
from bs4 import BeautifulSoup
from numpy import load, save,  asarray
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import networkx as nx
from scipy.sparse import csr_matrix
import warnings
warnings.filterwarnings("ignore")

In [3]:
tmp_dir = "Le/DLM-main/DLM-main/tmp"
data_dir = "Le/DLM-main/DLM-main/html_4000_files"

In [4]:
doc2vec = gensim.models.doc2vec.Doc2Vec.load("doc2vec.model")

In [5]:
from utils import generate_filename, convert, read_corpus

In [6]:
def callTag(tag, node):
    node_from = node
    node = node + 1

    for tag in tag.children:
        attr_node = 1
        if (tag.name is not None):
            nodes_list.append(convert([str(node_from), str(node)]))
            attr_list.append(str(node) + ",nname," + str(tag.name))
            attr_list.append(str(node) + ",value," + "")
            for attr in tag.attrs:
                nodes_list.append(convert([str(node), str(node) + "_" + str(attr_node)]))
                attr_list.append(str(node) + "_" + str(attr_node) + ",nname," + str(attr))
                attr_list.append(str(node) + "_" + str(attr_node) + ",value," + str(tag.get(attr)))
                attr_node = attr_node + 1
            callTag(tag, node)

In [7]:
def get_udst_inputs(url, rec_id, phishing_flag):
    node = 0
    f = codecs.open(Path(data_dir + '/' + generate_filename(rec_id)), 'r', encoding='utf-8', errors='ignore')
    soup = BeautifulSoup(f)
    callTag(soup, node)

    nodes_list.pop(0)
    attr_list.pop(1)
    attr_list.insert(1, '1, value, ' + url)

    G = nx.Graph()
    G.add_edges_from(nodes_list)

    A = nx.adjacency_matrix(G) # N*N Adjacency matrix

    df_features = pd.DataFrame(0.0, index=G.nodes(), columns=['nname', 'value'])

    attr_val_list = []
    for x in attr_list:
        y = x.split(',')
        if y[1] == "value":
            attr_val_list.append(y[2])

    for x in attr_list:
        y = x.split(',')
        test_corpus = list(read_corpus([y[2]], tokens_only=True))
        vector = doc2vec.infer_vector(test_corpus[0])
        # print(test_corpus[0])

        if y[1] == "nname":
            df_features.loc[y[0]]["nname"] = vector

        else:
            df_features.loc[y[0]]["value"] = vector

    X = df_features.values # N*d Feature Matrix
    # y = to_categorical(phishing_flag, num_classes=2).tolist() # Label
    y = phishing_flag

    X_ = np.array([np.array(ai, dtype=np.float32) for ai in X.tolist()])

    # url_token = url_tokenizer([url])[0]
    url_token = None

    return X_, A, y, url_token, url

In [8]:
# Delete empty HTML files
# data_type = 'Validation' # Train, Test, Validation
# df_te = pd.read_csv('Le/DLM-main/DLM-main/data1/'+data_type+'_Data.csv')
# exist_idx = []
# for idx, data in df_te.iterrows():
#     my_file = Path(data_dir + '/' + generate_filename(data['rec_id']))
#     if my_file.is_file():
#         if os.stat(data_dir + '/' + generate_filename(data['rec_id'])).st_size > 0:
#             exist_idx.append(idx)
        
# newtr = df_te.iloc[exist_idx,]
# newtr.to_csv('Le/DLM-main/DLM-main/data1/'+data_type+'_NewData.csv', index = False)

In [9]:
df_train = pd.read_csv('Le/DLM-main/DLM-main/data1/Train_NewData.csv')
df_val = pd.read_csv('Le/DLM-main/DLM-main/data1/Validation_NewData.csv')
df_test = pd.read_csv('Le/DLM-main/DLM-main/data1/Test_NewData.csv')
# exist_idx = []
# for idx, data in df_tr.iterrows():
#     my_file = Path(data_dir + '/' + generate_filename(data['rec_id']))
#     if my_file.is_file():
#         print(data['rec_id'], data['url'], data['result'], 'Y')
#     else:
#         print('=========================')

In [10]:
for _, data in df_test.iloc[3:4,:].iterrows():
    # node = 0
    nodes_list = []
    attr_list = []
    X, A, y, X_A, _ = get_udst_inputs(data['url'], data['rec_id'], data['result'])

In [11]:
X.shape, A.shape

((151, 2), (151, 151))

### Load data

In [12]:
%load_ext autoreload
%autoreload 2

In [13]:
# # data_dir = "data/URLdatasetX2_1.csv"
# data_dir = "data/URLdatasetX2_1sub10.csv"
# df = pd.read_csv(data_dir,index_col=0)
# # n_subsample = 3000 # all
# # smalldata = df.sample(n = n_subsample, random_state=1) #PC
# n_subsample = 'full'; smalldata = df;
# # get labels of urls
# labels = smalldata.iloc[:,-1].values
# from sklearn.preprocessing import LabelEncoder
# label_encoder = LabelEncoder()
# labels = label_encoder.fit_transform(labels)
smalldata = pd.concat([df_train, df_test])
labels = np.concatenate([df_train['result'], df_test['result']])

In [14]:
smalldata.head(2)

,rec_id,url,result
0,7,https://kluban.net,0
1,15,https://1ki174.com,0


In [15]:
import collections
counter = collections.Counter(labels)
counter, len(labels)

(Counter({0: 182, 1: 357}), 539)

### Conventional Models

In [16]:
from utils import extract_features

In [17]:
# get numerical and catergorical features
phish_url = []
for link in list(smalldata.iloc[:,1]):
    url_features = extract_features(link)
    phish_url.append(list(url_features.values())[1:])

In [18]:
phish_url_df = pd.DataFrame(phish_url, columns = list(url_features.keys())[1:])

In [19]:
phish_url_df.head(2)

,num_subdomains,contains_ip,path_length,num_path_segments,uses_https,file_extension,count_special_characters,count_non_alphanumeric_characters,TLD,count_obfuscated_characters,letter_ratio_in_url,digit_ratio_in_url,count_equals_in_url,NoOfAmpersandInURL,CharContinuationRate,ratio_obfuscated_characters,NoOfQMarkInURL
0,1,0,0,1,1,,4,4,net,0,0.777778,0.000000,0,0,0.111111,0.0,0
1,1,0,0,1,1,,4,4,com,0,0.555556,0.222222,0,0,0.111111,0.0,0


In [20]:
phish_url_df.iloc[:,5] = pd.Categorical(phish_url_df.iloc[:,5]).codes
phish_url_df.iloc[:,8] = pd.Categorical(phish_url_df.iloc[:,8]).codes

In [21]:
phish_url_df.head(2)

,num_subdomains,contains_ip,path_length,num_path_segments,uses_https,file_extension,count_special_characters,count_non_alphanumeric_characters,TLD,count_obfuscated_characters,letter_ratio_in_url,digit_ratio_in_url,count_equals_in_url,NoOfAmpersandInURL,CharContinuationRate,ratio_obfuscated_characters,NoOfQMarkInURL
0,1,0,0,1,1,0,4,4,50,0,0.777778,0.000000,0,0,0.111111,0.0,0
1,1,0,0,1,1,0,4,4,14,0,0.555556,0.222222,0,0,0.111111,0.0,0


In [22]:
# # test on URLs features
# run_ML(phish_url_df, labels, "URLdatasetX2", "manual")

In [23]:
## test on numerical URLs features
from utils import extract_numerical_features
phish_url = []
for link in list(smalldata.iloc[:,1]):
    url_features = extract_numerical_features(link)
    phish_url.append(list(url_features.values()))
run_ML(np.array(phish_url), labels, "Cuvation", "manual_numerical")

Run:  0 , fold:  0
Train freq:  [151, 280]
kNN, LightGBM, Run:  0 , fold:  1
Train freq:  [141, 290]
kNN, LightGBM, Run:  0 , fold:  2
Train freq:  [150, 281]
kNN, LightGBM, Run:  0 , fold:  3
Train freq:  [140, 291]
kNN, LightGBM, Run:  0 , fold:  4
Train freq:  [146, 286]
kNN, LightGBM, ['kNN', 'LightGBM']
[0.97 0.98]


In [24]:
np.random.seed(0)
n_samples = len(smalldata.index)
train_idx = range(len(df_train.index))
test_idx = list(set(list(range(n_samples))).difference(set(train_idx)))
#data_df = np.array(phish_url_df)
data_df = np.array(phish_url)
import lightgbm as lgb
model = lgb.LGBMClassifier(verbose=-1)
model.fit(data_df[train_idx], labels[train_idx])
y_predict=model.predict(data_df[test_idx]) 
print(f1_score(y_predict, labels[test_idx], average='macro'))

0.9753197622050082


### Extract graph features from URLs for PyG

In [25]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
# List of URLs
urls = list(smalldata['url'])
# Tokenization and N-grams Generation
# You can adjust ngram_range to extract different n-grams (e.g., (1, 1) for unigrams, (2, 2) for bigrams, etc.)
vectorizer = CountVectorizer(analyzer='char', ngram_range=(1, 4)) #5
X_counts = vectorizer.fit_transform(urls)
# TF-IDF Transformation
transformer = TfidfTransformer()
X_tfidf = transformer.fit_transform(X_counts)
# Extracted Features
feature_names = vectorizer.get_feature_names_out()
X_counts_data = X_counts.toarray() # not necessary
# Train lgb
model_lgb = lgb.LGBMClassifier(verbose=-1)
model_lgb.fit(X_counts_data[train_idx], labels[train_idx])
y_predict=model_lgb.predict(X_counts_data[test_idx]) 
print(f1_score(y_predict, labels[test_idx], average='macro'))
feature_imp_gain = pd.DataFrame(sorted(zip(model_lgb.booster_.feature_importance(importance_type='gain'),
                                           feature_names), reverse=True), columns=['Value', 'Feature'])
feature_imp_split = pd.DataFrame(sorted(zip(model_lgb.booster_.feature_importance(importance_type='split'),
                                            feature_names), reverse=True), columns=['Value', 'Feature'])
top_ngrams_features = list(set(list(feature_imp_gain.iloc[:200,1]) + list(feature_imp_split.iloc[:200,1])))
cv = CountVectorizer(analyzer='char', ngram_range=(1, 4))
cv.fit(top_ngrams_features)

1.0


CountVectorizer(analyzer='char', ngram_range=(1, 4))

In [26]:
def extract_feature_CountVectorizer(model, url):
    return model.transform([url]).toarray().flatten()

In [27]:
# extract_feature_CountVectorizer(cv, urls[0])

In [28]:
import requests
from bs4 import BeautifulSoup
from requests.exceptions import ConnectionError
import traceback
import logging

In [29]:
# return root and hyperlinks features
def get_graph_features_CountVectorizer(idx):
    url = smalldata.iloc[idx,1]
    root_feature = extract_feature_CountVectorizer(cv, url) # dict
    hyperlink_data = [list(root_feature)]
    try:    
        # find all hyperlinks
        rec_id = smalldata.iloc[idx, 0]
        f = codecs.open(Path(data_dir + '/' + generate_filename(rec_id)), 'r', encoding='utf-8', errors='ignore')
        soup = BeautifulSoup(f)
        urls = []
        count = 0;
        for link in soup.find_all('a'):
            # print(link.get('href'))
            weblink = link.get('href')
            if (weblink is not None) and ('http' in weblink):
                urls.append(weblink)
            count += 1
            if count > 50:
                break
        # extract numerical features in from hyperlinks
        if len(urls) > 0:
            for link in urls:
                try:
                    url_features = extract_feature_CountVectorizer(cv, link)
                    datalinkssss = list(url_features)
                    hyperlink_data.append(datalinkssss)
                except ValueError as ve:
                    # datalinkssss = list(np.zeros(15))#raw_graph_features
                    error_here = 1;
                # hyperlink_data.append(datalinkssss)
        else:
            # hyperlink_data.append(list(np.zeros(15)))#raw_graph_features
            error_here = 1;
    
    # except ConnectionError as e:
    #     # print("No rep", end = ',')
    #     # hyperlink_data.append(list(np.zeros(15))) #raw_graph_features
    #     error_here = 1; #v2
    except Exception as e:
        #logging.error(traceback.format_exc())
        error_here = 1
    
    return (idx,  hyperlink_data)

# Run HTML features

In [32]:
## Idea: Combine with manual features from HTMLs

In [47]:
# HTML offline
import requests
from bs4 import BeautifulSoup
import re
# import whois
from urllib.parse import urlparse
from collections import Counter

# Function to parse HTML content from a URL
def get_html_content(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise error if the response is bad
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL {url}: {e}")
        return None

# Function to extract HTML structure features
def extract_structure_features(soup):
    features = {}
    
    features['num_divs'] = len(soup.find_all('div'))
    features['num_scripts'] = len(soup.find_all('script'))
    features['num_forms'] = len(soup.find_all('form'))
    features['num_links'] = len(soup.find_all('a'))
    features['num_iframes'] = len(soup.find_all('iframe'))
    
    # Text to HTML ratio
    text_length = len(soup.get_text())
    html_length = len(str(soup))
    features['text_to_html_ratio'] = text_length / html_length if html_length > 0 else 0
    
    return features

# Function to extract text-based features
def extract_text_features(soup):
    features = {}
    
    # Keywords in meta tags
    meta_keywords = soup.find('meta', attrs={'name': 'keywords'})
    #features['meta_keywords'] = meta_keywords['content'] if meta_keywords else ''
    
    # Title tag content
    title = soup.title.string if soup.title else ''
    features['title_length'] = len(title)
    
    # Visible text length
    features['visible_text_length'] = len(soup.get_text())
    
    return features

# Function to extract link-based features
def extract_link_features(soup, base_url):
    features = {}
    
    all_links = soup.find_all('a', href=True)
    internal_links = [link for link in all_links if urlparse(link['href']).netloc == urlparse(base_url).netloc]
    external_links = [link for link in all_links if urlparse(link['href']).netloc != urlparse(base_url).netloc]
    
    features['num_internal_links'] = len(internal_links)
    features['num_external_links'] = len(external_links)
    
    return features

# Function to extract JavaScript-based features
def extract_js_features(soup):
    features = {}
    
    script_tags = soup.find_all('script')
    features['num_js_files'] = len(script_tags)
    
    # Suspicious JavaScript patterns
    suspicious_js_patterns = ['eval', 'document.write', 'window.location']
    features['suspicious_js_count'] = sum(
        any(p in script.get_text() for p in suspicious_js_patterns) for script in script_tags)
    
    return features

# Function to extract all features from a URL
def extract_features(url, rec_id):
#     html_content = get_html_content(url)
#     if not html_content:
#         return None
    
#     soup = BeautifulSoup(html_content, 'html.parser')
    
    f = codecs.open(Path(data_dir + '/' + generate_filename(rec_id)), 'r', encoding='utf-8', errors='ignore')
    soup = BeautifulSoup(f)
    
    # Extract all feature sets
    structure_features = extract_structure_features(soup)
    text_features = extract_text_features(soup)
    link_features = extract_link_features(soup, url)
    js_features = extract_js_features(soup)
    # domain_features = extract_domain_features(url)
    
    # Combine all features into a single dictionary
    features = {**structure_features, **text_features, **link_features, **js_features}
    
    return features

In [48]:
# # Example URLs for feature extraction
# urls = [
#     "https://dantri.com.vn"
# ]

# # Extract features from each URL
# features_list = []
# for url in urls:
#     features = extract_features(url)
#     if features:
#         features_list.append(features)

# # Print extracted features
# for idx, features in enumerate(features_list):
#     print(f"Features for URL {urls[idx]}:\n", features)

In [62]:
idx = 0
url = df_train.iloc[idx, 1]
rec_id = smalldata.iloc[idx, 0]
features = extract_features(url, rec_id)
print(list(features.values()))

[158, 17, 1, 129, 0, 0.049341527655838456, 5, 9554, 0, 129, 17, 0]


In [64]:
manual_html_features = []
for idx in range(len(smalldata.index)):
    url = smalldata.iloc[idx, 1]
    rec_id = smalldata.iloc[idx, 0]
    features = extract_features(url, rec_id)
    manual_html_features.append(list(features.values()))

In [66]:
run_ML(np.array(manual_html_features), labels, "Cuvation", "HTML")

Run:  0 , fold:  0
Train freq:  [151, 280]
kNN, LightGBM, Run:  0 , fold:  1
Train freq:  [141, 290]
kNN, LightGBM, Run:  0 , fold:  2
Train freq:  [150, 281]
kNN, LightGBM, Run:  0 , fold:  3
Train freq:  [140, 291]
kNN, LightGBM, Run:  0 , fold:  4
Train freq:  [146, 286]
kNN, LightGBM, ['kNN', 'LightGBM']
[0.55 0.64]


### Create dataset class for PyG

In [93]:
if False: #don't delete this
    graphs = []
    labels_list = []
    for i in range(len(smalldata.index)):
    # for i in range(2):
        nodes_list = []
        attr_list = []
        print(i, end = ',')
        X, A, y, X_A, _ = get_udst_inputs(smalldata.iloc[i, 1], smalldata.iloc[i, 0], smalldata.iloc[i, 2])

        child_id, source_id = A.nonzero()
        edge_index = torch.tensor([source_id,
                                   child_id], dtype=torch.long)
        x = torch.tensor(X, dtype=torch.float)
        y = torch.tensor([labels[i]], dtype=torch.int64)
        data = Data(x=x, edge_index=edge_index, y = y)
        graphs.append(data)
        labels_list.append(labels[idx])

In [35]:
import pickle
# with open("graph_html", "wb") as fp:   #Pickling
#     pickle.dump(graphs, fp)
# with open("graph_html_labels", "wb") as fp:   #Pickling
#     pickle.dump(labels_list, fp)
    
with open("graph_html", "rb") as fp:   # Unpickling
    graphs = pickle.load(fp)
with open("graph_html_labels", "rb") as fp:   # Unpickling
    labels_list = pickle.load(fp)

In [36]:
print("Train freq: ", [len(list(group)) for key, group in groupby(sorted(labels_list))])

Train freq:  [539]


In [37]:
import torch
from torch_geometric.data import Data, Dataset

class GraphClassificationDataset(Dataset):
    def __init__(self, graphs):
        self.graphs = graphs
        # self.labels = labels

    def __len__(self):
        return len(self.graphs)

    def __getitem__(self, idx):
        graph = self.graphs[idx]
        # label = self.labels[idx]
        return graph
    
    def get(): pass

    def len(): pass

In [38]:
dataset = GraphClassificationDataset(graphs)

In [39]:
print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.


Dataset: GraphClassificationDataset(539):
Number of graphs: 539
Number of features: 2
Number of classes: 2


In [40]:
# dataset = dataset.shuffle()
n_samples = len(dataset)
# np.random.seed(0) 
# train_idx = list(np.random.choice(list(range(n_samples)), int(0.8*n_samples), replace=False))
# test_idx = list(set(list(range(n_samples))).difference(set(train_idx)))
# train_idx[:10]

In [41]:
# train_dataset = dataset[:int(0.8*n_samples)]
# test_dataset = dataset[int(0.8*n_samples):]
train_dataset = [dataset[idx] for idx in train_idx]
test_dataset = [dataset[idx] for idx in test_idx]

In [42]:
len(train_dataset), len(test_dataset)

(402, 137)

In [43]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

### Build and train PyG

In [44]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool, global_max_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, dataset.num_classes)
        self.linconcat = Linear(2*hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

In [45]:
model = GCN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
        # print(data.x.shape)
        out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.

def test(loader):
    model.eval()
    correct = 0
    # for data in loader:  # Iterate in batches over the training/test dataset.
    #     out = model(data.x, data.edge_index, data.batch)  
    #     pred = out.argmax(dim=1)  # Use the class with highest probability.
    #     correct += int((pred == data.y).sum())  # Check against ground-truth labels.
    # return correct / len(loader.dataset)  # Derive ratio of correct predictions.
    true_labels = []
    pred_labels = []
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
        true_labels += data.y.tolist()
        pred_labels += pred.tolist()
        # print(pred_labels)
    return f1_score(true_labels, pred_labels, average='macro')

for epoch in range(0, 20):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:03d}, Train F1: {train_acc:.4f}, Test F1: {test_acc:.4f}')

Epoch: 000, Train F1: 0.3982, Test F1: 0.3991
Epoch: 010, Train F1: 0.3982, Test F1: 0.3991


In [46]:
# ['kNN', 'LightGBM'] min, max, avg of child features [0.82 0.92]

### GNN for dim. reduction

#### Net 1: 0.87, 0.88

In [47]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool, global_max_pool

class GCNdimReduce(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCNdimReduce, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin1 = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = self.lin1(x)
        x = F.dropout(x, p=0.3, training=self.training)
        x = self.lin2(x)
        
        return x
    
    def dimReduce(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        # x = x.relu()
        # x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # # 3. Apply a final classifier
        x = self.lin1(x)
        return x

#### Net 2

In [48]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool, global_max_pool

class GCNdimReduceV2(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCNdimReduceV2, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin1 = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
       
        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        x = F.dropout(x, p=0.5, training=self.training)
        # 3. Apply a final classifier
        x = self.lin1(x)
        x = x.relu()
        x = self.lin2(x)
        
        return x
    
    def dimReduce(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin1(x)
        return x

#### Test model

In [49]:
n_hidden_channels = 16
model = GCNdimReduce(hidden_channels=n_hidden_channels)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
        # print(data.x.shape)
        out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.

def test(loader):
    model.eval()
    correct = 0
    true_labels = []
    pred_labels = []
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
        true_labels += data.y.tolist()
        pred_labels += pred.tolist()
    return f1_score(true_labels, pred_labels, average='macro')

for epoch in range(0, 10):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    if epoch % 1 == 0:
        print(f'Epoch: {epoch:03d}, Train F1: {train_acc:.4f}, Test F1: {test_acc:.4f}')

Epoch: 000, Train F1: 0.3982, Test F1: 0.3991
Epoch: 001, Train F1: 0.3982, Test F1: 0.3991
Epoch: 002, Train F1: 0.3982, Test F1: 0.3991
Epoch: 003, Train F1: 0.3982, Test F1: 0.3991
Epoch: 004, Train F1: 0.3982, Test F1: 0.3991
Epoch: 005, Train F1: 0.3982, Test F1: 0.3991
Epoch: 006, Train F1: 0.3982, Test F1: 0.3991
Epoch: 007, Train F1: 0.3982, Test F1: 0.3991
Epoch: 008, Train F1: 0.3982, Test F1: 0.3991
Epoch: 009, Train F1: 0.3982, Test F1: 0.3991


#### Def

In [50]:
phish_url_vectorizer = []
for link in list(smalldata.iloc[:,1]):
    url_features = extract_feature_CountVectorizer(cv, link)
    phish_url_vectorizer.append(list(url_features))
# run_ML(concatGNN, labels, "URLdatasetX2", "concatGNN")

In [51]:
def train_model(all_data_loader, train_loader, test_loader, n_hidden_channels = 16, n_epoch=1, lr=0.001):
    # n_hidden_channels = 16
    model = GCNdimReduce(hidden_channels=n_hidden_channels)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss()

    def train():
        model.train()

        for data in train_loader:  # Iterate in batches over the training dataset.
            # print(data.x.shape)
            out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
            loss = criterion(out, data.y)  # Compute the loss.
            loss.backward()  # Derive gradients.
            optimizer.step()  # Update parameters based on gradients.
            optimizer.zero_grad()  # Clear gradients.

    def test(loader):
        model.eval()
        correct = 0
        true_labels = []
        pred_labels = []
        for data in loader:  # Iterate in batches over the training/test dataset.
            out = model(data.x, data.edge_index, data.batch)  
            pred = out.argmax(dim=1)  # Use the class with highest probability.
            correct += int((pred == data.y).sum())  # Check against ground-truth labels.
            true_labels += data.y.tolist()
            pred_labels += pred.tolist()
        return f1_score(true_labels, pred_labels, average='macro')

    for epoch in range(0, n_epoch):
        train()
        train_acc = test(train_loader)
        test_acc = test(test_loader)
        if epoch % 1 == 0:
            print(f'Epoch: {epoch:03d}, Train F1: {train_acc:.4f}, Test F1: {test_acc:.4f}')
    
    model.eval()
    dim_vec = torch.empty((0, n_hidden_channels), dtype=torch.float32)
    for data in all_data_loader:
        dim_x = model.dimReduce(data.x, data.edge_index, data.batch)
        dim_vec = torch.cat((dim_vec, dim_x), 0)
    return (dim_vec)

In [52]:
all_data_loader = DataLoader(dataset, batch_size=1, shuffle=False)
dim_vec = train_model(all_data_loader, train_loader, test_loader)
# model.eval()
# dim_vec = torch.empty((0, n_hidden_channels), dtype=torch.float32)
# # data = next(iter(test_loader))
# for data in all_data_loader:
#     dim_x = model.dimReduce(data.x, data.edge_index, data.batch)
#     dim_vec = torch.cat((dim_vec, dim_x), 0)

Epoch: 000, Train F1: 0.3982, Test F1: 0.3991


In [53]:
# # Combine hidden features with URL features
# concatGNN = np.concatenate((np.array(phish_url_vectorizer), dim_vec.detach().numpy()),axis=1)
# model_lgb = lgb.LGBMClassifier(verbose=-1)
# model_lgb.fit(concatGNN[train_idx], labels[train_idx])
# y_predict=model_lgb.predict(concatGNN[test_idx]) 
# print(f1_score(y_predict, labels[test_idx], average='macro'))

In [54]:
# # Use hidden feature for testing
# model_lgb = lgb.LGBMClassifier(verbose=-1)
# graph_embedding = dim_vec.detach().numpy()
# model_lgb.fit(graph_embedding[train_idx], labels[train_idx])
# y_predict=model_lgb.predict(graph_embedding[test_idx]) 
# print(f1_score(y_predict, labels[test_idx], average='macro'))

In [55]:
graphs[0].x.shape[1]

2

In [56]:
# Stats graph feature
n_features_counter = int(graphs[0].x.shape[1]);
html_hyperlink_features = np.zeros((smalldata.shape[0], 3*n_features_counter))
for idx in range(len(graphs)):
    # print(idx, hyper_np)
    hyper_np = np.array(graphs[idx].x)
    if hyper_np.shape[0] >= 2:
        html_hyperlink_features[idx, :] = np.hstack((hyper_np.min(axis=0),hyper_np.max(axis=0), hyper_np.mean(axis=0)))

In [57]:
concatGNN_graph = np.concatenate((np.array(phish_url_vectorizer), html_hyperlink_features),axis=1)
# concatGNN_graph = np.array(phish_url_vectorizer)

In [58]:
model_lgb2 = lgb.LGBMClassifier(verbose=-1)
model_lgb2.fit(concatGNN_graph[train_idx], labels[train_idx])
y_predict=model_lgb2.predict(concatGNN_graph[test_idx]) 
print(f1_score(y_predict, labels[test_idx], average='macro'))

1.0


In [59]:
def runGRAPHISH(train_idx, test_idx):
    stack_GNNs_graph = np.concatenate((np.array(phish_url_vectorizer), html_hyperlink_features),axis=1)
    # stack_GNNs_graph = np.array(phish_url_vectorizer)
    for i in range(10):
        np.random.seed(i) 
        # train_idx_new = list(np.random.choice(list(range(n_samples)), int(0.8*n_samples), replace=False))
        # train_idx_new = list(set(train_idx_new).difference(test_idx))
        train_idx_new = list(np.random.choice(train_idx, int(0.8*len(train_idx)), replace=False))
        print(train_idx_new[:5])
        train_dataset_new = [dataset[idx] for idx in train_idx_new]
        train_loader_new = DataLoader(train_dataset_new, batch_size=8, shuffle=True)
        n_hidden_channels = 2
        n_epoch = 1
        dim_vec_new = train_model(all_data_loader, train_loader_new, test_loader, n_hidden_channels, n_epoch)
        stack_GNNs_graph = np.concatenate((stack_GNNs_graph, dim_vec_new.detach().numpy()),axis=1)
    # Le: compute new method
    
    
    model_lgb2 = lgb.LGBMClassifier(verbose=-1)
    model_lgb2.fit(stack_GNNs_graph[train_idx], labels[train_idx])
    y_predict=model_lgb2.predict(stack_GNNs_graph[test_idx]) 
    y_proba=model_lgb2.predict_proba(stack_GNNs_graph[test_idx])[:,1]
    print(f1_score(y_predict, labels[test_idx], average='macro'))
    df_results = pd.DataFrame({'true_label': labels[test_idx], 'pred_label': y_predict, 'predict_proba': y_proba})
    return df_results

In [60]:
runGRAPHISH(train_idx, test_idx)

[199, 397, 59, 78, 65]
Epoch: 000, Train F1: 0.3977, Test F1: 0.3991
[162, 300, 400, 117, 80]
Epoch: 000, Train F1: 0.3989, Test F1: 0.3991
[25, 337, 53, 100, 190]
Epoch: 000, Train F1: 0.4067, Test F1: 0.3991
[248, 320, 81, 255, 217]
Epoch: 000, Train F1: 0.3955, Test F1: 0.3991
[184, 6, 61, 312, 100]
Epoch: 000, Train F1: 0.3977, Test F1: 0.3991
[52, 123, 282, 71, 222]
Epoch: 000, Train F1: 0.4000, Test F1: 0.3991
[53, 392, 61, 13, 373]
Epoch: 000, Train F1: 0.4067, Test F1: 0.3991
[248, 65, 120, 132, 380]
Epoch: 000, Train F1: 0.3989, Test F1: 0.3991
[59, 210, 97, 305, 194]
Epoch: 000, Train F1: 0.4045, Test F1: 0.3991
[57, 32, 17, 167, 2]
Epoch: 000, Train F1: 0.3955, Test F1: 0.3991
1.0


,true_label,pred_label,predict_proba
0,1,1,0.999985
1,1,1,0.999985
2,1,1,0.999985
3,1,1,0.999985
4,1,1,0.999985
...,...,...,...
132,1,1,0.999985
133,1,1,0.999985
134,1,1,0.999983
135,1,1,0.999979


# Hyperlinks data

In [61]:
import multiprocessing
multiprocessing.cpu_count()
n_cores = min(30, int(multiprocessing.cpu_count()-2))
n_cores

30

In [62]:
data_name_0123 = data_dir.split('/')[-1][:-4]
n_subsample = 100
data_name_0123

'html_4000_f'

In [63]:
data_name_0123 = data_dir.split('/')[-1][:-4]
# data_file = "data/raw_graph_features_v2.pickle" # first version 
data_file = "data/"+data_name_0123+str(n_subsample)+"v2_raw_graph_features_CountVectorizer.pickle" # first version 
my_file = Path(data_file)
if my_file.is_file():
    print("File exist! Load the data")
    with open(data_file, "rb") as fp:   # Unpickling
        results = pickle.load(fp)
else:
    print("File does not exist! Process the data")
    n_test_samples = int(smalldata.shape[0]) # how many link we want to test
    from joblib import Parallel, delayed
    results = Parallel(n_jobs=n_cores)(delayed(get_graph_features_CountVectorizer)(i) for i in range(n_test_samples)) # test on 100 links
    with open(data_file, "wb") as fp:   #Pickling
        pickle.dump(results, fp)

File exist! Load the data


In [64]:
data_file

'data/html_4000_f100v2_raw_graph_features_CountVectorizer.pickle'

In [65]:
len(results)

539

### Create dataset class for PyG

In [66]:
# Assume you have a list of graphs represented as Data objects and a corresponding list of labels
hyperlink_graphs = []
hyperlink_labels_list = []
for i in range(len(results)):
    idx, graph_feature = results[i]
    n_hyperlinks = len(graph_feature)-1
    child_id = [i+1 for i in range(n_hyperlinks)]
    source_id = list(np.zeros(n_hyperlinks).astype(int))
    # edges are connecting node 0 to nodes 1, 2, .., n_hpyterlinks
    edge_index = torch.tensor([source_id,
                               child_id], dtype=torch.long)
    x = torch.tensor(graph_feature, dtype=torch.float)
    y = torch.tensor([labels[idx]], dtype=torch.int64)
    data = Data(x=x, edge_index=edge_index, y = y)
    if n_hyperlinks > -1:
        hyperlink_graphs.append(data)
        hyperlink_labels_list.append(labels[idx])

In [67]:
print("Train freq: ", [len(list(group)) for key, group in groupby(sorted(hyperlink_labels_list))])

Train freq:  [182, 357]


In [68]:
hyperlink_dataset = GraphClassificationDataset(hyperlink_graphs)

In [69]:
print()
print(f'Dataset: {hyperlink_dataset}:')
print('====================')
print(f'Number of graphs: {len(hyperlink_dataset)}')
print(f'Number of features: {hyperlink_dataset.num_features}')
print(f'Number of classes: {hyperlink_dataset.num_classes}')

data = dataset[0]  # Get the first graph object.


Dataset: GraphClassificationDataset(539):
Number of graphs: 539
Number of features: 369
Number of classes: 2


In [70]:
# dataset = dataset.shuffle()
n_samples = len(hyperlink_dataset)
# np.random.seed(0) 
# train_idx = list(np.random.choice(list(range(n_samples)), int(0.8*n_samples), replace=False))
# test_idx = list(set(list(range(n_samples))).difference(set(train_idx)))
# train_idx[:10]

In [71]:
# train_dataset = dataset[:int(0.8*n_samples)]
# test_dataset = dataset[int(0.8*n_samples):]
hyperlink_train_dataset = [hyperlink_dataset[idx] for idx in train_idx]
hyperlink_test_dataset = [hyperlink_dataset[idx] for idx in test_idx]

In [72]:
len(hyperlink_train_dataset), len(hyperlink_test_dataset)

(402, 137)

In [73]:
from torch_geometric.loader import DataLoader

hyperlink_train_loader = DataLoader(hyperlink_train_dataset, batch_size=16, shuffle=True)
hyperlink_test_loader = DataLoader(hyperlink_test_dataset, batch_size=16, shuffle=False)

### Build and train PyG

In [74]:
# from torch.nn import Linear
# import torch.nn.functional as F
# from torch_geometric.nn import GCNConv
# from torch_geometric.nn import global_mean_pool, global_max_pool


# class GCN(torch.nn.Module):
#     def __init__(self, hidden_channels):
#         super(GCN, self).__init__()
#         torch.manual_seed(12345)
#         self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
#         self.conv2 = GCNConv(hidden_channels, hidden_channels)
#         self.conv3 = GCNConv(hidden_channels, hidden_channels)
#         self.lin = Linear(hidden_channels, dataset.num_classes)
#         self.linconcat = Linear(2*hidden_channels, dataset.num_classes)

#     def forward(self, x, edge_index, batch):
#         # 1. Obtain node embeddings 
#         x = self.conv1(x, edge_index)
#         x = x.relu()
#         x = self.conv2(x, edge_index)
#         x = x.relu()
#         x = self.conv3(x, edge_index)

#         # 2. Readout layer
#         x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

#         # 3. Apply a final classifier
#         x = F.dropout(x, p=0.5, training=self.training)
#         x = self.lin(x)
        
#         return x

In [75]:
# model = GCN(hidden_channels=64)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# criterion = torch.nn.CrossEntropyLoss()

# def train():
#     model.train()

#     for data in train_loader:  # Iterate in batches over the training dataset.
#         # print(data.x.shape)
#         out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
#         loss = criterion(out, data.y)  # Compute the loss.
#         loss.backward()  # Derive gradients.
#         optimizer.step()  # Update parameters based on gradients.
#         optimizer.zero_grad()  # Clear gradients.

# def test(loader):
#     model.eval()
#     correct = 0
#     # for data in loader:  # Iterate in batches over the training/test dataset.
#     #     out = model(data.x, data.edge_index, data.batch)  
#     #     pred = out.argmax(dim=1)  # Use the class with highest probability.
#     #     correct += int((pred == data.y).sum())  # Check against ground-truth labels.
#     # return correct / len(loader.dataset)  # Derive ratio of correct predictions.
#     true_labels = []
#     pred_labels = []
#     for data in loader:  # Iterate in batches over the training/test dataset.
#         out = model(data.x, data.edge_index, data.batch)  
#         pred = out.argmax(dim=1)  # Use the class with highest probability.
#         correct += int((pred == data.y).sum())  # Check against ground-truth labels.
#         true_labels += data.y.tolist()
#         pred_labels += pred.tolist()
#         # print(pred_labels)
#     return f1_score(true_labels, pred_labels, average='macro')

# for epoch in range(0, 20):
#     train()
#     train_acc = test(train_loader)
#     test_acc = test(test_loader)
#     if epoch % 10 == 0:
#         print(f'Epoch: {epoch:03d}, Train F1: {train_acc:.4f}, Test F1: {test_acc:.4f}')

In [76]:
# ['kNN', 'LightGBM'] min, max, avg of child features [0.82 0.92]

### GNN for dim. reduction

#### Net 1: 0.87, 0.88

In [77]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool, global_max_pool

class hyperlink_GCNdimReduce(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(hyperlink_GCNdimReduce, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(hyperlink_dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin1 = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, hyperlink_dataset.num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = self.lin1(x)
        x = F.dropout(x, p=0.3, training=self.training)
        x = self.lin2(x)
        
        return x
    
    def dimReduce(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        # x = x.relu()
        # x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # # 3. Apply a final classifier
        x = self.lin1(x)
        return x

#### Net 2

In [78]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool, global_max_pool

class hyperlink_GCNdimReduceV2(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(hyperlink_GCNdimReduceV2, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(hyperlink_dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin1 = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, hyperlink_dataset.num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
       
        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        x = F.dropout(x, p=0.5, training=self.training)
        # 3. Apply a final classifier
        x = self.lin1(x)
        x = x.relu()
        x = self.lin2(x)
        
        return x
    
    def dimReduce(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin1(x)
        return x

#### Test model

In [79]:
n_hidden_channels = 16
hyperlink_model = hyperlink_GCNdimReduce(hidden_channels=n_hidden_channels)
optimizer = torch.optim.Adam(hyperlink_model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

def train():
    hyperlink_model.train()

    for data in hyperlink_train_loader:  # Iterate in batches over the training dataset.
        # print(data.x.shape)
        out = hyperlink_model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.

def test(loader):
    hyperlink_model.eval()
    correct = 0
    true_labels = []
    pred_labels = []
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = hyperlink_model(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
        true_labels += data.y.tolist()
        pred_labels += pred.tolist()
    return f1_score(true_labels, pred_labels, average='macro')

for epoch in range(0, 10):
    train()
    train_acc = test(hyperlink_train_loader)
    test_acc = test(hyperlink_test_loader)
    if epoch % 1 == 0:
        print(f'Epoch: {epoch:03d}, Train F1: {train_acc:.4f}, Test F1: {test_acc:.4f}')

Epoch: 000, Train F1: 0.3982, Test F1: 0.3991
Epoch: 001, Train F1: 0.5324, Test F1: 0.4863
Epoch: 002, Train F1: 0.8981, Test F1: 0.9338
Epoch: 003, Train F1: 0.9307, Test F1: 0.9330
Epoch: 004, Train F1: 0.9457, Test F1: 0.9418
Epoch: 005, Train F1: 0.9671, Test F1: 0.9418
Epoch: 006, Train F1: 0.9539, Test F1: 0.9589
Epoch: 007, Train F1: 0.9862, Test F1: 0.9418
Epoch: 008, Train F1: 0.9889, Test F1: 0.9153
Epoch: 009, Train F1: 0.9861, Test F1: 0.9062


#### Def

In [80]:
phish_url_vectorizer = []
for link in list(smalldata.iloc[:,1]):
    url_features = extract_feature_CountVectorizer(cv, link)
    phish_url_vectorizer.append(list(url_features))
# run_ML(concatGNN, labels, "URLdatasetX2", "concatGNN")

In [81]:
def train_hyperlink_model(all_data_loader, train_loader, test_loader, n_hidden_channels = 16, n_epoch=1, lr=0.001):
    # n_hidden_channels = 16
    model = hyperlink_GCNdimReduce(hidden_channels=n_hidden_channels)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss()

    def train():
        model.train()

        for data in train_loader:  # Iterate in batches over the training dataset.
            # print(data.x.shape)
            out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
            loss = criterion(out, data.y)  # Compute the loss.
            loss.backward()  # Derive gradients.
            optimizer.step()  # Update parameters based on gradients.
            optimizer.zero_grad()  # Clear gradients.

    def test(loader):
        model.eval()
        correct = 0
        true_labels = []
        pred_labels = []
        for data in loader:  # Iterate in batches over the training/test dataset.
            out = model(data.x, data.edge_index, data.batch)  
            pred = out.argmax(dim=1)  # Use the class with highest probability.
            correct += int((pred == data.y).sum())  # Check against ground-truth labels.
            true_labels += data.y.tolist()
            pred_labels += pred.tolist()
        return f1_score(true_labels, pred_labels, average='macro')

    for epoch in range(0, n_epoch):
        train()
        train_acc = test(train_loader)
        test_acc = test(test_loader)
        if epoch % 1 == 0:
            print(f'Epoch: {epoch:03d}, Train F1: {train_acc:.4f}, Test F1: {test_acc:.4f}')
    
    model.eval()
    dim_vec = torch.empty((0, n_hidden_channels), dtype=torch.float32)
    for data in all_data_loader:
        dim_x = model.dimReduce(data.x, data.edge_index, data.batch)
        dim_vec = torch.cat((dim_vec, dim_x), 0)
    return (dim_vec)

In [82]:
hyperlink_all_data_loader = DataLoader(hyperlink_dataset, batch_size=1, shuffle=False)
hyperlink_dim_vec = train_hyperlink_model(hyperlink_all_data_loader, hyperlink_train_loader, hyperlink_test_loader)
# model.eval()
# dim_vec = torch.empty((0, n_hidden_channels), dtype=torch.float32)
# # data = next(iter(test_loader))
# for data in all_data_loader:
#     dim_x = model.dimReduce(data.x, data.edge_index, data.batch)
#     dim_vec = torch.cat((dim_vec, dim_x), 0)

Epoch: 000, Train F1: 0.3982, Test F1: 0.3991


In [83]:
# concatGNN = np.concatenate((np.array(phish_url_vectorizer), hyperlink_dim_vec.detach().numpy()),axis=1)
# model_lgb = lgb.LGBMClassifier(verbose=-1)
# model_lgb.fit(concatGNN[train_idx], labels[train_idx])
# y_predict=model_lgb.predict(concatGNN[test_idx]) 
# print(f1_score(y_predict, labels[test_idx], average='macro'))

In [84]:
# model_lgb = lgb.LGBMClassifier(verbose=-1)
# graph_embedding = hyperlink_dim_vec.detach().numpy()
# model_lgb.fit(graph_embedding[train_idx], labels[train_idx])
# y_predict=model_lgb.predict(graph_embedding[test_idx]) 
# print(f1_score(y_predict, labels[test_idx], average='macro'))

In [85]:
# Stats graph feature
idx, vec = results[0]; vec = np.array(vec); n_features_counter = int(vec.shape[1]);
hyperlink_features = np.zeros((smalldata.shape[0], 3*n_features_counter))
for idx, hyper_np in results:
    # print(idx, hyper_np)
    hyper_np = np.array(hyper_np)
    if hyper_np.shape[0] >= 2:
        hyperlink_features[idx, :] = np.hstack((hyper_np.min(axis=0),hyper_np.max(axis=0), hyper_np.mean(axis=0)))
    # hyperlink_features[idx, :] = hyper_np

In [86]:
concatGNN_graph = np.concatenate((np.array(phish_url_vectorizer), hyperlink_features),axis=1)
# concatGNN_graph = np.concatenate((np.array(phish_url_vectorizer), hyperlink_features,  dim_vec.detach().numpy()),axis=1)

In [87]:
model_lgb2 = lgb.LGBMClassifier(verbose=-1)
model_lgb2.fit(concatGNN_graph[train_idx], labels[train_idx])
y_predict=model_lgb2.predict(concatGNN_graph[test_idx]) 
print(f1_score(y_predict, labels[test_idx], average='macro'))

1.0


In [88]:
def runHyperlinkGRAPHISH(train_idx, test_idx):
    stack_GNNs_graph = np.concatenate((np.array(phish_url_vectorizer), hyperlink_features),axis=1)
    for i in range(10):
        np.random.seed(i) 
        # train_idx_new = list(np.random.choice(list(range(n_samples)), int(0.8*n_samples), replace=False))
        # train_idx_new = list(set(train_idx_new).difference(test_idx))
        train_idx_new = list(np.random.choice(train_idx, int(0.8*len(train_idx)), replace=False))
        print(train_idx_new[:5])
        train_dataset_new = [hyperlink_dataset[idx] for idx in train_idx_new]
        train_loader_new = DataLoader(train_dataset_new, batch_size=8, shuffle=True)
        n_hidden_channels = 2
        n_epoch = 1
        dim_vec_new = train_hyperlink_model(hyperlink_all_data_loader, train_loader_new, hyperlink_test_loader, n_hidden_channels, n_epoch)
        stack_GNNs_graph = np.concatenate((stack_GNNs_graph, dim_vec_new.detach().numpy()),axis=1)
    # Le: compute new method
    
    
    model_lgb2 = lgb.LGBMClassifier(verbose=-1)
    model_lgb2.fit(stack_GNNs_graph[train_idx], labels[train_idx])
    y_predict=model_lgb2.predict(stack_GNNs_graph[test_idx]) 
    y_proba=model_lgb2.predict_proba(stack_GNNs_graph[test_idx])[:,1]
    print(f1_score(y_predict, labels[test_idx], average='macro'))
    df_results = pd.DataFrame({'true_label': labels[test_idx], 'pred_label': y_predict, 'predict_proba': y_proba})
    return df_results

In [89]:
runHyperlinkGRAPHISH(train_idx, test_idx)

[199, 397, 59, 78, 65]
Epoch: 000, Train F1: 0.7186, Test F1: 0.8737
[162, 300, 400, 117, 80]
Epoch: 000, Train F1: 0.3360, Test F1: 0.4439
[25, 337, 53, 100, 190]
Epoch: 000, Train F1: 0.5794, Test F1: 0.8188
[248, 320, 81, 255, 217]
Epoch: 000, Train F1: 0.7188, Test F1: 0.9130
[184, 6, 61, 312, 100]
Epoch: 000, Train F1: 0.6859, Test F1: 0.8981
[52, 123, 282, 71, 222]
Epoch: 000, Train F1: 0.4950, Test F1: 0.6194
[53, 392, 61, 13, 373]
Epoch: 000, Train F1: 0.2934, Test F1: 0.4804
[248, 65, 120, 132, 380]
Epoch: 000, Train F1: 0.6955, Test F1: 0.8981
[59, 210, 97, 305, 194]
Epoch: 000, Train F1: 0.6735, Test F1: 0.8907
[57, 32, 17, 167, 2]
Epoch: 000, Train F1: 0.4266, Test F1: 0.5486
0.991773254068336


,true_label,pred_label,predict_proba
0,1,1,0.999982
1,1,1,0.999979
2,1,1,0.999968
3,1,1,0.999985
4,1,1,0.999982
...,...,...,...
132,1,1,0.999964
133,1,1,0.999983
134,1,1,0.999942
135,1,1,0.999983


In [90]:
def runHTMLHyperlinkGRAPHISH(train_idx, test_idx):
    stack_GNNs_graph = np.concatenate((np.array(phish_url_vectorizer), hyperlink_features),axis=1)
    for i in range(10):
        np.random.seed(i) 
        # train_idx_new = list(np.random.choice(list(range(n_samples)), int(0.8*n_samples), replace=False))
        # train_idx_new = list(set(train_idx_new).difference(test_idx))
        train_idx_new = list(np.random.choice(train_idx, int(0.8*len(train_idx)), replace=False))
        print(train_idx_new[:5])
        train_dataset_new = [hyperlink_dataset[idx] for idx in train_idx_new]
        train_loader_new = DataLoader(train_dataset_new, batch_size=8, shuffle=True)
        n_hidden_channels = 2
        n_epoch = 1
        dim_vec_new = train_hyperlink_model(hyperlink_all_data_loader, train_loader_new, hyperlink_test_loader, n_hidden_channels, n_epoch)
        stack_GNNs_graph = np.concatenate((stack_GNNs_graph, dim_vec_new.detach().numpy()),axis=1)
    # HTML: compute new method
    for i in range(10):
        np.random.seed(i) 
        # train_idx_new = list(np.random.choice(list(range(n_samples)), int(0.8*n_samples), replace=False))
        # train_idx_new = list(set(train_idx_new).difference(test_idx))
        train_idx_new = list(np.random.choice(train_idx, int(0.8*len(train_idx)), replace=False))
        print(train_idx_new[:5])
        train_dataset_new = [dataset[idx] for idx in train_idx_new]
        train_loader_new = DataLoader(train_dataset_new, batch_size=8, shuffle=True)
        n_hidden_channels = 2
        n_epoch = 1
        dim_vec_new = train_model(all_data_loader, train_loader_new, test_loader, n_hidden_channels, n_epoch)
        stack_GNNs_graph = np.concatenate((stack_GNNs_graph, dim_vec_new.detach().numpy()),axis=1)
    
    model_lgb2 = lgb.LGBMClassifier(verbose=-1)
    model_lgb2.fit(stack_GNNs_graph[train_idx], labels[train_idx])
    y_predict=model_lgb2.predict(stack_GNNs_graph[test_idx]) 
    y_proba=model_lgb2.predict_proba(stack_GNNs_graph[test_idx])[:,1]
    print(f1_score(y_predict, labels[test_idx], average='macro'))
    df_results = pd.DataFrame({'true_label': labels[test_idx], 'pred_label': y_predict, 'predict_proba': y_proba})
    return df_results

In [91]:
runHTMLHyperlinkGRAPHISH(train_idx, test_idx)

[199, 397, 59, 78, 65]
Epoch: 000, Train F1: 0.7186, Test F1: 0.8737
[162, 300, 400, 117, 80]
Epoch: 000, Train F1: 0.3360, Test F1: 0.4439
[25, 337, 53, 100, 190]
Epoch: 000, Train F1: 0.5794, Test F1: 0.8188
[248, 320, 81, 255, 217]
Epoch: 000, Train F1: 0.7188, Test F1: 0.9130
[184, 6, 61, 312, 100]
Epoch: 000, Train F1: 0.6859, Test F1: 0.8981
[52, 123, 282, 71, 222]
Epoch: 000, Train F1: 0.4950, Test F1: 0.6194
[53, 392, 61, 13, 373]
Epoch: 000, Train F1: 0.2934, Test F1: 0.4804
[248, 65, 120, 132, 380]
Epoch: 000, Train F1: 0.6955, Test F1: 0.8981
[59, 210, 97, 305, 194]
Epoch: 000, Train F1: 0.6735, Test F1: 0.8907
[57, 32, 17, 167, 2]
Epoch: 000, Train F1: 0.4266, Test F1: 0.5486
[199, 397, 59, 78, 65]
Epoch: 000, Train F1: 0.3977, Test F1: 0.3991
[162, 300, 400, 117, 80]
Epoch: 000, Train F1: 0.3989, Test F1: 0.3991
[25, 337, 53, 100, 190]
Epoch: 000, Train F1: 0.4067, Test F1: 0.3991
[248, 320, 81, 255, 217]
Epoch: 000, Train F1: 0.3955, Test F1: 0.3991
[184, 6, 61, 312, 100]

,true_label,pred_label,predict_proba
0,1,1,0.999974
1,1,1,0.999975
2,1,1,0.999978
3,1,1,0.999985
4,1,1,0.999975
...,...,...,...
132,1,1,0.999979
133,1,1,0.999982
134,1,1,0.999947
135,1,1,0.999980


In [92]:
# data_set_save = 'Test_NewData'
# n_loops = 1; n_folds = 5;
# base_dir = 'comparision_results/html/test1'
# approach = 'GRAPHISHS'
# data_set = data_dir[5:-4]
# from sklearn.model_selection import KFold
# from sklearn.model_selection import StratifiedKFold
# for i in range(n_loops):
#     cv = KFold(n_splits=n_folds, shuffle=True, random_state = i)
#     # print("Use stratifired fold")
#     # cv = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state = i)
#     for fold, (train_idx, test_idx) in enumerate(cv.split(phish_url_vectorizer)):
#         path_dir = base_dir +'/' + data_set_save + '_run_'+str(i)+'_'+ 'fold_'+str(fold)+'_'+approach
#         print('Run: ', i, ', fold: ', fold)
#         # X_train = X[train_idx]
#         # X_test = X[test_idx]
#         # y_train = y[train_idx]
#         # y_test = y[test_idx]
#         df = runGRAPHISH(train_idx, test_idx)
#         path_dir = base_dir +'/' + data_set_save + '_run_'+str(i)+'_'+ 'fold_'+str(fold)+'_'+approach
#         df.to_csv(path_dir + '_GRAPHISH_labels.csv', index=False)

In [2]:
A = [8, 9, 3, 4, 2]
n = len(A)

In [3]:
for i in range(n):
    for j in range(i+1, n):
        if A[i] > A[j]:
            # Doi cho A[i] va A[j]
            temp = A[j]
            A[j] = A[i]
            A[i] = temp

In [4]:
A

[2, 3, 4, 8, 9]